In [1]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
sys.path.append('../code/')
import d2lzh_pytorch as d2l
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(corpus_indices,char_to_idx,idx_to_char,vocab_size)=d2l.load_data_jay_lyrics()

In [2]:
def one_hot(x,n_class,dtype=torch.float32):
    x=x.long()
    res=torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res

In [3]:
x=torch.tensor([0,2])
one_hot(x,vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [4]:
def to_onehot(X,n_class):
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]

In [5]:
X=torch.arange(10).view(2,5)
inputs=to_onehot(X,vocab_size)
print(len(inputs),len(inputs),inputs[0].shape)

5 5 torch.Size([2, 1027])


In [6]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)

def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    W_xh=_one((num_inputs,num_hiddens))
    W_hh=_one((num_hiddens,num_hiddens))
    b_h=torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])

will use cuda


In [7]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device), )

In [8]:
def rnn(inputs,state,params):
    W_xh,W_hh,b_h,W_hq,b_q=params
    H, =state
    outputs=[]
    for X in inputs:
        H=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [9]:
state=init_rnn_state(X.shape[0],num_hiddens,device)
inputs=to_onehot(X.to(device),vocab_size)
params=get_params()
outputs,state_new=rnn(inputs,state,params)
print(len(outputs),outputs[0].shape,state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


In [10]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,
              idx_to_char,char_to_idx):
    state=init_rnn_state(1,num_hiddens,device)
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X=to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state)=rnn(X,state,params)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [11]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开正兵死卷花雨咒半离思'

In [40]:
def grad_clipping(params,theta,device):
    norm = torch.tensor([0.0],device=device)
    for param in params:
        norm +=(param.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm >theta:
        for param in params:
            param.grad.data*=(theta/norm)

In [44]:
def train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,device,corpus_indices,
                          idx_to_char,char_to_idx,is_random_iter,num_epochs,num_steps,lr,clipping_theta,
                          batch_size,pred_period,pred_len,prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn=d2l.data_iter_consecutive
    params=get_params()
    loss=nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state=init_rnn_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter:
            if is_random_iter:
                state=init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
            inputs=to_onehot(X,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [47]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 500, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [48]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 68.158929, time 0.15 sec
 - 分开 你想你 想你么 我想要你想 我不要 想你的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
 - 不分开 你想你 想你么 我想要你想 我不要 想你的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
epoch 100, perplexity 10.333759, time 0.15 sec
 - 分开 一颗两步 在小村空的溪边 默默等待 娘子我有著在猜 一壶好酒 一颗心动的溪边 默默等待 娘小的脚 
 - 不分开永 我不要再想你 不知不觉 你已经离开我 不知不觉 我已了这节奏 后知后觉 我已经好生我 不知不觉 
epoch 150, perplexity 2.879997, time 0.15 sec
 - 分开 一只用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼无兮 快
 - 不分开扫 我后能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 200, perplexity 1.565496, time 0.15 sec
 - 分开 一只到 干手箱著 沙漠的中 全人家  不果有痛 一句种痛 在一场中 不敢去碰 没有笑 痛不知轻重 
 - 不分开扫 不道开过 全小我早已经猜透看透不想多说 只是我怕眼你的定爸 停格在容后 不要 神殿的征为的公 时
epoch 250, perplexity 1.297625, time 0.15 sec
 - 分开 一只到着心泣猜母的心窗 夕阳斜斜映在斑驳的砖墙 铺著榉木板的屋内还弥漫 姥姥当年酿的豆瓣酱 我对著
 - 不分开期 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子B 瞎 说一去看一个会老地许伤   在来了最
epoch 300, perplexity 1.208773, time 0.15 sec
 - 分开不  是情来着看酒已布我 没和你 别沉我抬起头 有话去对医药箱说 别怪我 别怪我 说你怎么 在漠伦 
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在吹我 已诉上有容  不爱要飞 
epoch 350, perplexity 1.155410, time 0.15 sec

In [49]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 55.700220, time 0.15 sec
 - 分开 我想要这 你小了外 如果我人 你不了空 如果我人 你不了空 如果我人 你不了空 如果我人 你不了空
 - 不分开 我想我这你的让我 你着我有你你我的可 就不我 别怪我有三 我不要再想 我不能再想 我不能再想 我不
epoch 100, perplexity 6.484616, time 0.15 sec
 - 分开 我想要这样牵着你的手不放开 爱可不可以简简单单下 我说定再想 我不 我不 我不能 爱情走的太快就像
 - 不分开柳 你已经离开我 不知不觉 漫天用双截棍 哼哼哈兮 快使我有轻功 飞檐走壁 快人我有轻功 飞檐走壁 
epoch 150, perplexity 2.019279, time 0.15 sec
 - 分开 对候的 旧时空中停留 所有人看着我 抛物线进球 单手过没有喝水也能活 脑袋瓜有一点秀逗 猎物死了它
 - 不分开柳 你已经离开我 不知不觉 我跟经这节奏 后知后觉 又过了一个秋 后知后觉 快使用双截棍 哼哼哈兮 
epoch 200, perplexity 1.296519, time 0.15 sec
 - 分开 问候我 谁情神枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅头 这故事 告诉我 印地安的传说 
 - 不分开柳 你已经离开我 后知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 250, perplexity 1.177499, time 0.15 sec
 - 分开 问候的 旧时空中我有 有却啊 娘不是你不着球 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久
 - 不分开觉 你已经离开我听错弄就是不著你 刻子黑色每日武不住 脸懂 你的黑色幽默 想通 却又再考倒我 说散 
epoch 300, perplexity 1.128638, time 0.15 sec
 - 分开 问候我 谁是神枪手 巫师 他念念 有词的 对酋长下诅咒 还我骷髅你 这故事 告诉我 印地安的传说 
 - 不分开柳 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 350, perplexity 1.101180, time 0.15 sec
